In [10]:
#!pip install "yfinance[nospam,repair]"

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
from datetime import datetime
import streamlit as st
import json
import re

In [57]:
def parse_german_date(date_string):
    if pd.isna(date_string):
        return pd.NaT
    try:
        german_months = {
            'Jan': 1, 'Feb': 2, 'Mär': 3, 'Apr': 4, 'Mai': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Okt': 10, 'Nov': 11, 'Dez': 12
        }
        day, month, year = date_string.split()
        month_num = german_months[month]
        return datetime(2000 + int(year), month_num, int(day))
    except (ValueError, KeyError):
        return pd.NaT  # Return Not-a-Time for any parsing errors

In [60]:
res = requests.get(f"https://www.eqs-news.com/wp/wp-admin/admin-ajax.php?lang=de&action=fetch_realtime_events_data&recordsFrom[0][api_type]=events&recordsFrom[0][category]=future&pageLimit=10000&pageNo=1&additional[is_new]=false&additional[mode]=append&loadFrom=mysql&time=")
if (res.status_code == 200):
    soup = BeautifulSoup(res.text, 'html.parser')
    events = []  
    event_divs = soup.find_all('div', class_='event')
    
    for event in event_divs:
        event_data = {}
        event_data['company'] = event.find('h4', class_='event__company').get_text(strip=True)
        event_data['title'] = event.find('p', class_='event__title').get_text(strip=True)
        #event_data['location'] = event.find('div', class_='event__location').get_text(strip=True) if event.find('div', class_='event__location') else None
        date = event.find('p', class_='event__date').get_text(strip=True)
        month_year = event.find('p', class_='event__month-year').get_text(strip=True)
        event_data['date'] = f"{date} {month_year}"
        #event_data['company_url'] = event['data-events-company-url']
        #event_data['company_share_url'] = event['data-events-company-share-url']
        #event_data['uuid'] = event['data-events-uuid']
        events.append(event_data)
    
    df = pd.DataFrame(events)
    df['converted_dates'] = df['date'].apply(parse_german_date)
    df.drop(columns=['date'], inplace=True)
    
df

,company,title,converted_dates
0,Intevac,Analystenveranstaltung,2024-08-05
1,WashTec AG,Veröffentlichung Halbjahresfinanzbericht,2024-08-05
2,Infineon Technologies AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2024-08-05
3,Aurubis AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2024-08-05
4,Crescent Energy Company,,2024-08-06
...,...,...,...
1077,IVU Traffic Technologies AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2025-11-20
1078,SCHOTT Pharma AG & Co. KGaA,Veröffentlichung Jahresfinanzbericht,2025-12-10
1079,All for One Group SE,Analystenveranstaltung,2025-12-15
1080,All for One Group SE,Bilanzpressekonferenz,2025-12-15


In [13]:
df.to_excel("EqsLoaderPlayground.xlsx", index=False)

In [62]:
st.dataframe(df)

2024-08-05 21:43:18.770 
  command:

    streamlit run D:\zail\eqsloader\eqsloader\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [95]:
res = requests.get(f"https://seekingalpha.com/api/v3/news?filter[category]=market-news%3A%3Anotable-calls&filter[since]=0&filter[until]=0&isMounting=true&page[size]=200&page[number]=1", 
                   headers={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
                             "accept-language": "de,en-US;q=0.9,en;q=0.8", 
                             "accept-encoding": "gzip, deflate, br, zstd" })


def extract_stock_abbr(content):
    match = re.search(r'\b[A-Z]{1,5}\b', content)
    if match:
        return match.group(0)
    return "N/A"

def convert_publish_date(date_str):
    # Extract current year
    current_year = datetime.now().year
    # Convert date string to datetime object
    date_without_year = datetime.strptime(date_str, "%a, %b. %d")
    # Return datetime object with current year
    return date_without_year.replace(year=current_year)


if (res.status_code == 200):
    data = json.loads(res.text)
    
    # Extract relevant information
    extracted_data = []
    for item in data["data"]:
        title = item["attributes"]["title"]
        content = item["attributes"]["content"]
        publish_date_str = datetime.strptime(item["attributes"]["publishOn"], "%Y-%m-%dT%H:%M:%S%z").strftime("%a, %b. %d")
        publish_date = convert_publish_date(publish_date_str)
        extracted_data.append([title, publish_date])
    
    # Create a pandas DataFrame
    df = pd.DataFrame(extracted_data, columns=["Title", "Publish Date"])
    print(res.text)
    
df

{"data":[{"id":"4135127","type":"news","attributes":{"publishOn":"2024-08-06T12:22:16-04:00","isLockedPro":false,"commentCount":6,"gettyImageUrl":"https://static.seekingalpha.com/cdn/s3/uploads/getty_images/1337403704/image_1337403704.jpg","videoPreviewUrl":null,"videoDuration":null,"themes":{"us":{"id":326,"path":"/","slug":"us","title":null,"sasource":"","non_theme":true},"technology":{"id":17994,"path":"/market-news/technology","slug":"technology","title":"Tech","sasource":"theme_breadcrumb","non_theme":false},"large-cap":{"id":115402,"path":"/","slug":"large-cap","title":null,"sasource":"","non_theme":true},"notable-calls":{"id":600890,"path":"/","slug":"notable-calls","title":null,"sasource":"","non_theme":true},"news-metered":{"id":614465,"path":"/","slug":"news-metered","title":null,"sasource":"","non_theme":true},"consumer":{"id":614789,"path":"/market-news/consumer","slug":"consumer","title":"Consumer ","sasource":"theme_breadcrumb","non_theme":false}},"title":"Street still pr

,Title,Stock Abbreviation,Publish Date
0,Street still processing the impact of Google's...,KEYB,2024-08-06
1,Jefferies thinks Netflix might raise prices by...,NFLX,2024-08-05
2,Morgan Stanley stays 'overweight' on Disney ah...,NYSE,2024-08-05
3,Lightshed assumes coverage on Reddit with 'buy...,NYSE,2024-08-05
4,Udemy rating cut at BofA after Q2 results; sha...,UDMY,2024-08-01
5,Top Street analysts weigh in on Meta's solid Q...,META,2024-08-01
6,AvidXchange downgraded to Neutral at UBS on lo...,AVDX,2024-08-01
7,Street weighs in on Pinterest after company wa...,NYSE,2024-07-31
8,Loop stays bullish on Disney stock for 'most c...,NYSE,2024-07-30
9,Morgan Stanley sticks to rating on WBD after N...,WBD,2024-07-29


In [92]:
from openai import OpenAI

MAX = 25
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": "You are an expert in financial news analysis. Extract the sentiment (buy/hold/sell) from the title of financial news articles, provide a brief reason for the sentiment, and identify the stock abbreviation. Add new columns for Sentiment, Reason and Abbreviation. Output the JSON again that can be used as pandas dataframe."
    },
    {
      "role": "user",
      "content": f"""
        {df[["Title"]][:MAX].to_json()}
      """
    }
  ],
  max_tokens=150 * MAX,
)

In [96]:
cleaned_response = response.choices[0].message.content.strip('```json\n').strip('```')
cleaned_response
data = json.loads(cleaned_response)
df_news = pd.DataFrame(data)
df_news

In [102]:
df_merged = pd.merge(df, df_news, on='Title', how='left')
df_merged

,Title,Publish Date,Sentiment,Reason,Abbreviation
0,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...,GOOGL
1,Jefferies thinks Netflix might raise prices by...,2024-08-05,buy,Potential price increase due to strong content...,NFLX
2,Morgan Stanley stays 'overweight' on Disney ah...,2024-08-05,buy,Positive outlook ahead of Q3 earnings.,DIS
3,Lightshed assumes coverage on Reddit with 'buy...,2024-08-05,buy,Positive growth predictions for Reddit.,REDDIT
4,Udemy rating cut at BofA after Q2 results; sha...,2024-08-01,sell,Lower rating due to disappointing Q2 results.,UDMY
5,Top Street analysts weigh in on Meta's solid Q...,2024-08-01,hold,Positive earnings report suggests stability.,META
6,AvidXchange downgraded to Neutral at UBS on lo...,2024-08-01,sell,Downgrade due to cautious guidance.,AVID
7,Street weighs in on Pinterest after company wa...,2024-07-31,sell,Concerns over revenue decline and currency iss...,PINS
8,Loop stays bullish on Disney stock for 'most c...,2024-07-30,buy,Strong risk/reward outlook for Disney.,DIS
9,Morgan Stanley sticks to rating on WBD after N...,2024-07-29,hold,Concerns about the impact of lost NBA rights.,WBD


In [103]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
import yfinance as yf

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass


session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [114]:
buy_abbreviations = df_merged[df_merged['Sentiment'] == 'buy']['Abbreviation']
buy_abbreviations

abbreviation_string = ' '.join(buy_abbreviations.tolist())
abbreviation_string

'NFLX DIS REDDIT DIS GOOGL MSGE SPOT SNAP RIO WEBTOON'

In [115]:
tickers = yf.Tickers(abbreviation_string, session=session)

In [125]:
buy_abbreviations

1        NFLX
2         DIS
3      REDDIT
8         DIS
10      GOOGL
16       MSGE
18       SPOT
20       SNAP
21        RIO
22    WEBTOON
Name: Abbreviation, dtype: object

In [138]:
results = []
for index, row in buy_abbreviations.items():
    res = tickers.tickers[row].history(period='5d')
    res['Abbreviation'] = row
    results.append(res)
    
# access each ticker using (example)
#tickers.tickers['MSFT'].info
#tickers.tickers['AAPL'].history(period="1mo")
#tickers.tickers['GOOG'].actions
#data = yf.download("SPY AAPL", period="1mo")

                                 Open        High         Low       Close  \
Date                                                                        
2024-07-31 00:00:00-04:00  626.570007  633.599976  622.580017  628.349976   
2024-08-01 00:00:00-04:00  629.250000  646.710022  617.609985  624.849976   
2024-08-02 00:00:00-04:00  620.979980  625.270020  608.349976  613.640015   
2024-08-05 00:00:00-04:00  587.039978  604.559998  587.039978  598.549988   
2024-08-06 00:00:00-04:00  601.114990  622.450012  600.630005  609.570007   

                            Volume  Dividends  Stock Splits Abbreviation  
Date                                                                      
2024-07-31 00:00:00-04:00  3134900        0.0           0.0         NFLX  
2024-08-01 00:00:00-04:00  3720600        0.0           0.0         NFLX  
2024-08-02 00:00:00-04:00  3523000        0.0           0.0         NFLX  
2024-08-05 00:00:00-04:00  4044200        0.0           0.0         NFLX  
2024-08-06

REDDIT: No data found, symbol may be delisted


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume, Abbreviation]
Index: []
                                Open       High        Low      Close  \
Date                                                                    
2024-07-31 00:00:00-04:00  93.790001  94.839996  93.639999  93.690002   
2024-08-01 00:00:00-04:00  93.860001  93.949997  92.599998  93.050003   
2024-08-02 00:00:00-04:00  92.160004  92.199997  89.000000  89.570000   
2024-08-05 00:00:00-04:00  86.889999  89.559998  86.419998  87.779999   
2024-08-06 00:00:00-04:00  88.230003  91.909698  87.959999  89.970001   

                             Volume  Dividends  Stock Splits Abbreviation  
Date                                                                       
2024-07-31 00:00:00-04:00  10241800        0.0           0.0          DIS  
2024-08-01 00:00:00-04:00   9263600        0.0           0.0          DIS  
2024-08-02 00:00:00-04:00  15433600        0.0           0.0          DIS  
2024-08-05 00:0

WEBTOON: No data found, symbol may be delisted


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume, Abbreviation]
Index: []


[                                 Open        High         Low       Close  \
 Date                                                                        
 2024-07-31 00:00:00-04:00  626.570007  633.599976  622.580017  628.349976   
 2024-08-01 00:00:00-04:00  629.250000  646.710022  617.609985  624.849976   
 2024-08-02 00:00:00-04:00  620.979980  625.270020  608.349976  613.640015   
 2024-08-05 00:00:00-04:00  587.039978  604.559998  587.039978  598.549988   
 2024-08-06 00:00:00-04:00  601.114990  622.450012  600.630005  609.570007   
 
                             Volume  Dividends  Stock Splits Abbreviation  
 Date                                                                      
 2024-07-31 00:00:00-04:00  3134900        0.0           0.0         NFLX  
 2024-08-01 00:00:00-04:00  3720600        0.0           0.0         NFLX  
 2024-08-02 00:00:00-04:00  3523000        0.0           0.0         NFLX  
 2024-08-05 00:00:00-04:00  4044200        0.0           0.0         NFL

In [139]:
tickers.tickers["NFLX"].history(period='5d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-07-31 00:00:00-04:00,626.570007,633.599976,622.580017,628.349976,3134900,0.0,0.0
2024-08-01 00:00:00-04:00,629.250000,646.710022,617.609985,624.849976,3720600,0.0,0.0
2024-08-02 00:00:00-04:00,620.979980,625.270020,608.349976,613.640015,3523000,0.0,0.0
2024-08-05 00:00:00-04:00,587.039978,604.559998,587.039978,598.549988,4044200,0.0,0.0
2024-08-06 00:00:00-04:00,601.114990,622.450012,600.630005,609.570007,4286101,0.0,0.0


In [145]:
result_df = pd.concat(results, ignore_index=False)
result_df['Date'] = result_df.index
result_df

C:\Users\lukas\AppData\Local\Temp\ipykernel_19124\2587111559.py:1: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  result_df = pd.concat(results, ignore_index=False)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Abbreviation,Adj Close,Date
Date,,,,,,,,,,
2024-07-31 00:00:00-04:00,626.570007,633.599976,622.580017,628.349976,3134900.0,0.0,0.0,NFLX,NaN,2024-07-31 00:00:00-04:00
2024-08-01 00:00:00-04:00,629.250000,646.710022,617.609985,624.849976,3720600.0,0.0,0.0,NFLX,NaN,2024-08-01 00:00:00-04:00
2024-08-02 00:00:00-04:00,620.979980,625.270020,608.349976,613.640015,3523000.0,0.0,0.0,NFLX,NaN,2024-08-02 00:00:00-04:00
2024-08-05 00:00:00-04:00,587.039978,604.559998,587.039978,598.549988,4044200.0,0.0,0.0,NFLX,NaN,2024-08-05 00:00:00-04:00
2024-08-06 00:00:00-04:00,601.114990,622.450012,600.630005,609.570007,4286101.0,0.0,0.0,NFLX,NaN,2024-08-06 00:00:00-04:00
2024-07-31 00:00:00-04:00,93.790001,94.839996,93.639999,93.690002,10241800.0,0.0,0.0,DIS,NaN,2024-07-31 00:00:00-04:00
2024-08-01 00:00:00-04:00,93.860001,93.949997,92.599998,93.050003,9263600.0,0.0,0.0,DIS,NaN,2024-08-01 00:00:00-04:00
2024-08-02 00:00:00-04:00,92.160004,92.199997,89.000000,89.570000,15433600.0,0.0,0.0,DIS,NaN,2024-08-02 00:00:00-04:00
2024-08-05 00:00:00-04:00,86.889999,89.559998,86.419998,87.779999,19103200.0,0.0,0.0,DIS,NaN,2024-08-05 00:00:00-04:00


In [146]:
df_final = pd.merge(result_df, df_merged, on='Abbreviation', how='right')
df_final

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Abbreviation,Adj Close,Date,Title,Publish Date,Sentiment,Reason
0,173.240005,174.250000,170.009995,171.539993,25729100.0,0.0,0.0,GOOGL,NaN,2024-07-31 00:00:00-04:00,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...
1,170.250000,174.050003,168.880005,170.759995,24531400.0,0.0,0.0,GOOGL,NaN,2024-08-01 00:00:00-04:00,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...
2,166.440002,168.509995,164.669998,166.660004,29130100.0,0.0,0.0,GOOGL,NaN,2024-08-02 00:00:00-04:00,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...
3,155.500000,164.429993,154.929993,159.250000,53630700.0,0.0,0.0,GOOGL,NaN,2024-08-05 00:00:00-04:00,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...
4,159.330002,160.570007,156.410004,158.289993,48521527.0,0.0,0.0,GOOGL,NaN,2024-08-06 00:00:00-04:00,Street still processing the impact of Google's...,2024-08-06,hold,Uncertainty regarding the antitrust suit's imp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,"UBS sees more headroom for Teradyne, hikes PT ...",2024-06-18,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,LittelFuse stock sparks after bullish Baird ra...,2024-05-17,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,"Ibotta catches Wall Street's attention, gets c...",2024-05-13,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Outlook Therapeutics shares rise after 'buy' r...,2024-03-27,NaN,NaN
